# Setting


> Imports



In [1]:
import os
import math
import numpy as np
from numpy import array
from numpy import hstack
import pandas as pd
import xarray as xr
import pickle
import tensorflow as tf
import seaborn as sns

import networkx as nx

import keras
from keras import layers, losses, regularizers
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras.models import Model, load_model
from keras.layers import Input, Dense, Flatten, Reshape, LeakyReLU, Cropping2D, RepeatVector, TimeDistributed, LSTM

from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, roc_curve, auc, precision_recall_curve

import matplotlib as mpl
from matplotlib import pyplot as plt
%matplotlib inline



> Connect to Google drive


In [2]:
from google.colab import drive
drive.mount('/content/drive')

os.chdir('/content/drive/My Drive/')
os.getcwd()


Mounted at /content/drive


'/content/drive/My Drive'

In [3]:
gb_df = pd.read_csv('JULES/gb_data.csv', index_col=0)
gb_data = gb_df.values
print(gb_data.shape)

scaler = preprocessing.MinMaxScaler()
gb_data = scaler.fit_transform(gb_data)
gb_data = np.array(gb_data, dtype = np.float32)

# 2
gb_df_2 = pd.read_csv('JULES/gb_data2.csv', index_col=0)
gb_data_2 = gb_df_2.values
print(gb_data_2.shape)

scaler_2 = preprocessing.MinMaxScaler()
gb_data_2 = scaler_2.fit_transform(gb_data_2)
gb_data_2 = np.array(gb_data_2, dtype = np.float32)

# 3
gb_df_3 = pd.read_csv('JULES/gb_data3.csv', index_col=0)
gb_data_3 = gb_df_3.values
print(gb_data_3.shape)

scaler_3 = preprocessing.MinMaxScaler()
gb_data_3 = scaler_3.fit_transform(gb_data_3)
gb_data_3 = np.array(gb_data_3, dtype = np.float32)

# 4
gb_df_4 = pd.read_csv('JULES/gb_data5.csv', index_col=0)
gb_data_4 = gb_df_4.values
print(gb_data_4.shape)

scaler_4 = preprocessing.MinMaxScaler()
gb_data_4 = scaler_4.fit_transform(gb_data_4)
gb_data_4 = np.array(gb_data_4, dtype = np.float32)

# 5
test_df = pd.read_csv('JULES/gb_data4.csv', index_col=0)
gb_data_test = test_df.values
print(gb_data_test.shape)

# Please note since the CAE-LSTM paper employed gb_data4 as test dataset
# For consistency we had adopted this, despite the impact on testing result is minimum

test_scaler = preprocessing.MinMaxScaler()
gb_data_test = test_scaler.fit_transform(gb_data_test)
gb_data_test = np.array(gb_data_test, dtype = np.float32)


gb_data = np.reshape(gb_data, (360, 112, 192, 1))
gb_data_2 = np.reshape(gb_data_2, (360, 112, 192, 1))
gb_data_3 = np.reshape(gb_data_3, (360, 112, 192, 1))
gb_data_4 = np.reshape(gb_data_4, (360, 112, 192, 1))
gb_data_test = np.reshape(gb_data_test, (360, 112, 192, 1))

# load the data
tair_df = pd.read_csv('JULES/tair.csv', index_col=0)
tair_data = tair_df.values
tair_data[np.isnan(tair_data)] = 0
min_max_scaler_tair = preprocessing.MinMaxScaler()
tair_data = min_max_scaler_tair.fit_transform(tair_data)
tair_data = np.array(tair_data, dtype = np.float32)

tair_data = np.reshape(tair_data, (360, 144, 192, 1))
tair_data = tair_data[:, 27:-5, :, :]

rain_df = pd.read_csv('JULES/rain.csv', index_col=0)
rain_data = rain_df.values
rain_data[np.isnan(rain_data)] = -0.001
min_max_scaler_rain = preprocessing.MinMaxScaler()
rain_data = min_max_scaler_rain.fit_transform(rain_data)
rain_data = np.array(rain_data, dtype = np.float32)

rain_data = np.reshape(rain_data, (360, 144, 192, 1))
rain_data = rain_data[:, 27:-5, :, :]

qair_df = pd.read_csv('JULES/qair.csv', index_col=0)
qair_data = qair_df.values
qair_data[np.isnan(qair_data)] = -0.001
min_max_scaler_qair = preprocessing.MinMaxScaler()
qair_data = min_max_scaler_qair.fit_transform(qair_data)
qair_data = np.array(qair_data, dtype = np.float32)

qair_data = np.reshape(qair_data, (360, 144, 192, 1))
qair_data = qair_data[:, 27:-5, :, :]

lightning_df = pd.read_csv('JULES/lightning.csv', index_col=0)
lightning_data = lightning_df.values
min_max_scaler_lightning = preprocessing.MinMaxScaler()
lightning_data = min_max_scaler_lightning.fit_transform(lightning_data)
lightning_data = np.array(lightning_data, dtype = np.float32)

lightning_data = np.reshape(lightning_data, (360, 144, 192, 1))
lightning_data = lightning_data[:, 27:-5, :, :]

(360, 21504)
(360, 21504)
(360, 21504)
(360, 21504)
(360, 21504)


In [4]:
gb_joint_data_1 = np.concatenate((gb_data, tair_data, rain_data, qair_data, lightning_data), axis = 3)
gb_joint_data_2 = np.concatenate((gb_data_2, tair_data, rain_data, qair_data, lightning_data), axis = 3)
gb_joint_data_3 = np.concatenate((gb_data_3, tair_data, rain_data, qair_data, lightning_data), axis = 3)
gb_joint_data_4 = np.concatenate((gb_data_4, tair_data, rain_data, qair_data, lightning_data), axis = 3)


test_joint_data = np.concatenate((gb_data_test, tair_data, rain_data, qair_data, lightning_data), axis = 3)

train_joint_data = np.concatenate((gb_joint_data_1, gb_joint_data_2, gb_joint_data_3, gb_joint_data_4), axis = 0)
print(train_joint_data.shape)
print(test_joint_data.shape)

(1440, 112, 192, 5)
(360, 112, 192, 5)


In [5]:
np.save('JULES/GCN_LSTM_Paper/dataset/uncompressed_train_data_v2.npy', train_joint_data)
np.save('JULES/GCN_LSTM_Paper/dataset/uncompressed_test_data_v2.npy', test_joint_data)